## Agile Hardware Design
***
# Lighting Chisel Tour for Verilog Experts

<img src="../resource/logo.svg" alt="agile hardware design logo" style="float:right"/>

## Prof. Scott Beamer
### sbeamer@ucsc.edu

## [CSE 228A](https://classes.soe.ucsc.edu/cse228a/Winter23/)

## Plan for Today

* Motivate the need for generators
* Chisel Overview
* Simple Chisel example
* Simple RISC-V in Chisel
* More Sophisticated Chisel Example
* Surprisingly concise switch in Chisel

## Loading The Chisel Library Into a Notebook

In [ ]:
interp.load.module(os.Path(s"${System.getProperty("user.dir")}/../resource/chisel_deps.sc"))

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test

## Increase Productivity Through Reuse

### Q: What is the fastest hardware block to develop?

### A: Hardware components you get to reuse!

* Don't keep reimplementing similar (or even same) components

* Spend your time on what is novel/unique for your design

* **Import caveat:** reused component needs to do _right thing_ and be _correct_

## HW Generators Increase Reusability

### Why not reuse? Existing component doesn't do what you need

* Generators are more _flexible_, and thus more _reusable_
* Generator can custom tailor the component for your design
* _Open-source_ generators also address when component exists but is proprietary
  * Generators make open-source more attractive, since flexibility grows the size of community that benefits from it
  * Larger community => larger amortization of design effort

### Key Concept: think of design _generation_ as another step in tool flow
* Write programs that design hardware for you (under controlled circumstances)
* Examples: [Genesys 2](https://github.com/StanfordVLSI/Genesis2/wiki) & [BaseJump STL](https://github.com/bespoke-silicon-group/basejump_stl)

### Chisel (Constructing Hardware In a Scala Embedded Language)

* Domain-specific language embedded in Scala
* Leverage best of object-oriented & functional programming
* Enables creation of design generators in a single language
* Well matched abstractions to constructable hardware
* Of next generation languages, has significant adoption
* Other features: abstract data types, bulk connections, width inference, standard library, multi-clock support

<img src="../01-intro/images/chisel_logo.svg" alt="chisel logo" style="float:right"/>


## Scala Language Summary

<img src="../01-intro/images/scala-spiral.png" alt="scala logo" style="float:right;width: 100px;"/>

_**Language Features**_
* Object oriented with strong static type system
* Native support for functional programing
* Runs on top of the JVM (and can interoperate with Java binaries)

_**Rationale for Using Scala**_
* Great support for implementing embedded domain-specific languages (DSL), e.g. Chisel
* Object oriented and functional features help make great _generators_
* Type system and included standard library’s collections

_**Syntax Curiosities**_
* No semicolons, declarations are typically immutable (`val`)
* Types go second (e.g. `myInt: Int`) and are often inferred
* Tries to catch many potential errors at compile time


## First Chisel Module

In [ ]:
class MyXOR extends Module {
    val io = IO(new Bundle {
        val a   = Input(Bool())
        val b   = Input(Bool())
        val c   = Output(Bool())
    })
    io.c := io.a ^ io.b
}

<img src="../02-hello/images/myxor.svg" alt="MyXOR schematic"  style="width:80%;margin:auto"/>

## Looking At Generated Design

In [ ]:
println(getVerilog(new MyXOR))

In [ ]:
visualize(() => new MyXOR)

## ChiselTest Execution

In [ ]:
test(new MyXOR()) { x =>
    x.io.a.poke(0.B)
    x.io.b.poke(0.B)
    x.io.c.expect(0.B)  // 0 ^ 0

    x.io.a.poke(0.B)
    x.io.b.poke(1.B)
    x.io.c.expect(1.B)  // 0 ^ 1

    x.io.a.poke(1.B)
    x.io.b.poke(0.B)
    x.io.c.expect(1.B)  // 1 ^ 0

    x.io.a.poke(1.B)
    x.io.b.poke(1.B)
    x.io.c.expect(0.B)  // 1 ^ 1
}

## Chisel Tool Flow (Frontend)

<img src="../02-hello/images/frontend.svg" alt="Chisel frontend" style="width:80%;margin:auto"/>

* The generated Circuit (`.fir` file) is a specific design instance, and it can be passed off to a _backend_ for simulation or implementation

## Chisel Tool Flow (Backend)


<img src="../02-hello/images/backend.svg" alt="Chisel backend" style="width:75%;margin:auto"/>

## Scala Values Are References to Chisel Objects

* Our generators are simply instantiating Chisel objects and connecting them together
  * Scala program allows us to control which objects & connections
* The connect operator (`:=`) assigns output of right hand side to input of left hand side
* Can use Scala references to name intermediate results

In [ ]:
class MyXOR extends Module {
    val io = IO(new Bundle {
        val a   = Input(Bool())
        val b   = Input(Bool())
        val c   = Output(Bool())
    })
    val myGate = io.a ^ io.b
    io.c := myGate
}
// println(getVerilog(new MyXOR))

<img src="../03-combo/images/xorRef.svg" alt="XOR with Scala references" style="width:55%;margin:auto"/>

# Implementing Parameterized Counter in Chisel

* Simple example, should use Counter from standard library

In [ ]:
class MyCounter(maxVal: Int) extends Module {
    val io = IO(new Bundle {
        val en  = Input(Bool())
        val out = Output(UInt())
    })
    val count = RegInit(0.U(log2Ceil(maxVal+1).W))
    when (io.en) {
        when (count < maxVal.U) {
            count := count + 1.U
        } .otherwise {
            count := 0.U
        }
    }
    io.out := count
}

# DinoCPU - Simple RISC-V Core

<img src="https://github.com/jlpteaching/dinocpu/raw/main/dino-128.png" alt="dinocpu logo" style="float:right;width: 128px;"/>

* **[DinoCPU GitHub](https://github.com/jlpteaching/dinocpu)**
* Simple RISC-V core for teaching from UC Davis
* Great demonstration of straightforward Chisel
* Also contains some parameterization





## Templated Queue

In [ ]:
class MyQueueV7[T <: chisel3.Data](numEntries: Int, gen: T, pipe: Boolean=true) extends Module {
    val io = IO(new Bundle {
        val enq = Flipped(Decoupled(gen))
        val deq = Decoupled(gen)
    })
    require(numEntries > 1)
    val entries = Mem(numEntries, gen)
    val enqIndex = Counter(numEntries)
    val deqIndex = Counter(numEntries)
    val maybeFull = RegInit(false.B)
    val indicesEqual = enqIndex.value === deqIndex.value
    val empty = indicesEqual && !maybeFull
    val full = indicesEqual && maybeFull
    if (pipe)
        io.enq.ready := !full || io.deq.ready
    else
        io.enq.ready := !full
    io.deq.valid := !empty
    io.deq.bits := entries(deqIndex.value)
    when (io.deq.fire =/= io.enq.fire) {
        maybeFull := io.enq.fire
    }
    when (io.deq.fire) {
        deqIndex.inc()
    }
    when (io.enq.fire) {
        entries(enqIndex.value) := io.enq.bits
        enqIndex.inc()
    }
}

## Example Crossbar in Chisel

* Connects `numIns` input ports to `numOuts` output ports
  * All ports are `Decoupled`

<img src="../08-arbit/images/xbar.svg" alt="xbar schematic" style="width:30%;margin-left:auto;margin-right:auto"/>

## Example Crossbar Implementation (1/2)

In [ ]:
class Message(numOuts: Int, length: Int) extends Bundle {
    val addr = UInt(log2Ceil(numOuts+1).W)
    val data = UInt(length.W)
}

class XBarIO(numIns: Int, numOuts: Int, length: Int) extends Bundle {
    val in  = Vec(numIns, Flipped(Decoupled(new Message(numOuts, length))))
    val out = Vec(numOuts, Decoupled(new Message(numOuts, length)))
}

## Example Crossbar Implementation (2/2)

In [ ]:
class XBar(numIns: Int, numOuts: Int, length: Int) extends Module {
    val io = IO(new XBarIO(numIns, numOuts, length))
    val arbs = Seq.fill(numOuts)(Module(new RRArbiter(new Message(numOuts, length), numIns)))
    for (ip <- 0 until numIns) {
        val inReadys = Wire(Vec(numOuts, Bool()))
        for (op <- 0 until numOuts) {
            inReadys(op) := arbs(op).io.in(ip).ready
        }
        io.in(ip).ready := inReadys.asUInt.orR
    }
    for (op <- 0 until numOuts) {
        for (ip <- 0 until numIns) {
            arbs(op).io.in(ip).bits <> io.in(ip).bits
            arbs(op).io.in(ip).valid := io.in(ip).valid && (io.in(ip).bits.addr === op.U)
        }
        io.out(op) <> arbs(op).io.out
    }
}

// println(getVerilog(new XBar(2,1,8)))

# Succinct Combinational Switch

In [ ]:
class CombSwitch(numIns: Int, numOuts: Int, w: Int) extends Module {
    val io = IO(new Bundle {
        val ins = Input(Vec(numIns, UInt(w.W)))
        val selects = Input(Vec(numOuts, UInt(w.W)))
        val outs = Output(Vec(numOuts, UInt(w.W)))
    })
    val cases = (0 until numIns).map(_.U).zip(io.ins)
    io.outs.zip(io.selects) foreach {
        case(o,s) => o := MuxLookup(s,DontCare,cases)
    }
}
// println(getVerilog(new CombSwitch(2,2,8)))

## Where to Go for More

* [Agile Hardware Design course](https://classes.soe.ucsc.edu/cse228a/Winter23/)
* [Chisel Homepage](https://www.chisel-lang.org)
* [Chisel Bootcamp](https://github.com/freechipsproject/chisel-bootcamp)
* [Hardware Systems Collective (at UCSC)](https://hsc.ucsc.edu)
* [Awesome Hardware Description Languages](https://github.com/drom/awesome-hdl)
* [Pyrope guest lecture by Prof. Jose Renau](https://www.youtube.com/watch?v=U4OX2tfiobE)